I will use spacy to transform words to vectors, and subsequently experiment with word arithmetic.  

In [25]:
# !python -m spacy download en_core_web_lg;

In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')

We will quantify the distance between two vectorized words using the cosine similarity:

In [3]:
from scipy import spatial
cos_sim = lambda v1,v2: 1 - spatial.distance.cosine(v1,v2)

Trying the word arithmetics for "actress", "male" and "female". The idea is, hopefully "actrees" - "female" + "male" results in "actor" among the top results.

In [4]:
# Vectorizing the words:

actress = nlp.vocab['actress'].vector
male = nlp.vocab['male'].vector
female = nlp.vocab['female'].vector

In [5]:
resultant = actress - female + male

We will now find top 10 most similar words to the above word vector:

In [6]:
similarities =[]

for word in nlp.vocab.vectors:
  word = nlp.vocab[word]
  if word.has_vector:
    if word.is_lower:
      if word.is_alpha:
        similarity = cos_sim(resultant, word.vector)
        similarities.append((word, similarity))

Sorting them to get the closest words according to this metric first:

In [7]:
similarities = sorted(similarities, key = lambda item: -item[1])

The following 10 words are the closest to "actress":

In [8]:
print([x[0].text for x in similarities[:10]])

['actress', 'actresses', 'actor', 'seductress', 'singer', 'male', 'benefactress', 'actos', 'directress', 'actonel']


Defining a function that takes in 3 word vectors a, b, and c and outputs a top-ten reseult of a-b+c arithmetic:

In [9]:
def resultant(a,b,c):
  cos_sim = lambda v_1,v_2: 1 - spatial.distance.cosine(v_1,v_2)
  v1 = nlp.vocab[a].vector
  v2 = nlp.vocab[b].vector
  v3 = nlp.vocab[c].vector
  resultant = v1-v2+v3
  similarities =[]
  for word in nlp.vocab.vectors:
    word = nlp.vocab[word]
    if word.has_vector:
      if word.is_lower:
       if word.is_alpha:
          similarity = cos_sim(resultant, word.vector)
          similarities.append((word, similarity))
  similarities = sorted(similarities, key = lambda item: -item[1])
  print([x[0].text for x in similarities[:10]])

In [10]:
resultant('actress','female','male')

['actress', 'actresses', 'actor', 'seductress', 'singer', 'male', 'benefactress', 'actos', 'directress', 'actonel']


In [11]:
resultant('king','man','woman')

['king', 'kings', 'princes', 'consort', 'princeling', 'monarch', 'princelings', 'princesses', 'prince', 'kingship']


# VADER sentiment analysis

In [12]:
import nltk

In [13]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Initializing sentiment analyzer object:

In [15]:
sid = SentimentIntensityAnalyzer()

In [16]:
review = 'the restaurant is very clean and our waiter was pleasant. The food was a little cold but overall, I enjoyed it.'

In [17]:
score = sid.polarity_scores(review)

In [18]:
score

{'neg': 0.0, 'neu': 0.65, 'pos': 0.35, 'compound': 0.8223}

Defining a function that takes in a review and returns a score of "Positive",
"Negative" or "Neutral":

In [19]:
def review_rating(string):
  score = sid.polarity_scores(string)
  if score['compound']<0:
    print('Negative')
  elif score['compound']>0:
    print("Positive")
  else:
    print("Neutral")

In [20]:
review_rating(review)

Positive


A couple of literary descriptions and their sentiment rating:

"The Brothers Karamazov": The mountains rose up sharply on all sides, their jagged peaks capped with pristine white snow that gleamed in the crisp alpine air. Down in the valley, a crystalline lake lay still and serene, reflecting the majesty of the towering peaks above.

In [21]:
Brothers_Karamazov = "The mountains rose up sharply on all sides, their jagged peaks capped with pristine white snow that gleamed in the crisp alpine air. Down in the valley, a crystalline lake lay still and serene, reflecting the majesty of the towering peaks above."

In [22]:
review_rating(Brothers_Karamazov)

Positive


"The Lord of the Rings": Upon its outward marges under the westward mountains Mordor was a dying land, but it was not yet dead. And here things still grew, harsh, twisted, bitter, struggling for life. In the glens of the Morgai on the other side of the valley low scrubby trees lurked and clung, coarse grey grass-tussocks fought with the stones, and withered mosses crawled on them; and everywhere great writhing, tangled brambles spawned. Some had long stabbing thorns, some hooked barbs that rent like knives. The sullen shrivelled leaves of a past year hung on them, grating and rattling in the sad airs, but their maggot-ridden buds were only just opening. Flies, dun or grey, or black, marked like orcs with a red eye-shaped blotch, buzzed and stung; and above the briar-thickets clouds of hungry midges danced and reeled.

In [23]:
Description_of_Mordor ="Upon its outward marges under the westward mountains Mordor was a dying land, but it was not yet dead. And here things still grew, harsh, twisted, bitter, struggling for life. In the glens of the Morgai on the other side of the valley low scrubby trees lurked and clung, coarse grey grass-tussocks fought with the stones, and withered mosses crawled on them; and everywhere great writhing, tangled brambles spawned. Some had long stabbing thorns, some hooked barbs that rent like knives. The sullen shrivelled leaves of a past year hung on them, grating and rattling in the sad airs, but their maggot-ridden buds were only just opening. Flies, dun or grey, or black, marked like orcs with a red eye-shaped blotch, buzzed and stung; and above the briar-thickets clouds of hungry midges danced and reeled."

In [24]:
review_rating(Description_of_Mordor)

Negative
